<a href="https://colab.research.google.com/github/Pranitha-59/AICTE-Internship/blob/main/FINAL_%20PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import callbacks
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
import json


In [ ]:
from google.colab import files
import zipfile
import os

uploaded = files.upload()


zip_filename = next((name for name in uploaded if name.endswith('.zip')), None)

if zip_filename is None:
    raise FileNotFoundError("Please upload a ZIP file containing the dataset.")


with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("/content/")


extracted_dir = None
for root, dirs, files in os.walk("/content/"):
    for dir_name in dirs:
        if "TrashType" in dir_name:  # match partially in case of minor name mismatch
            extracted_dir = os.path.join(root, dir_name)
            break
    if extracted_dir:
        break

if not extracted_dir or not tf.io.gfile.exists(extracted_dir):
    raise FileNotFoundError("Could not locate the extracted dataset directory.")



In [ ]:
image_size = (260, 260)
batch_size = 32
seed = 42

train_ds = tf.keras.utils.image_dataset_from_directory(
    extracted_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    extracted_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("✅ Dataset loaded with classes:", class_names)



In [ ]:
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

def preprocess(image, label):
    return preprocess_input(image), label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)



In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()



In [ ]:
autotune = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(autotune)
val_ds = val_ds.cache().prefetch(autotune)

def preprocess(image, label):
    return preprocess_input(image), label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)


In [ ]:
base_model = EfficientNetV2B2(include_top=False, input_shape=(260, 260, 3), weights="imagenet")
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)




In [ ]:
model.save("efficientnet_garbage_classifier.keras")
with open("class_names.json", "w") as f:
    json.dump(class_names, f)



In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training vs Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training vs Validation Loss')
plt.tight_layout()
plt.show()




In [ ]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)
test_ds = test_ds.cache().prefetch(tf.data.AUTOTUNE)

model.evaluate(test_ds)


In [ ]:
y_true, y_pred = [], []
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import gradio as gr
from PIL import Image

def predict(img):
    img = img.resize((260, 260))
    img = np.array(img)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    preds = model.predict(img)
    return {class_names[i]: float(preds[0][i]) for i in range(len(class_names))}

gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Garbage Classifier",
    description="Upload an image of waste to classify it as paper, plastic, metal, etc."
).launch()
